In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from helper import DateToOrdinal

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score

import xgboost as xgb

from sklearn.model_selection import GridSearchCV

In [2]:
data_folder = "data"
filename = "train.csv"
df = pd.read_csv(os.path.join("..", data_folder, filename))#, parse_dates=True, index_col="date")

In [3]:
def create_lagged_features(df, value, lags):
    for lag in range(1, lags + 1):
        df[f't-{lag}'] = df[value].shift(lag)
    df.dropna(inplace=True)
    return df

In [4]:
df = df[["orders", "date", "warehouse"]]

In [5]:
df

,orders,date,warehouse
0,6895.0,2020-12-05,Prague_1
1,6584.0,2020-12-06,Prague_1
2,7030.0,2020-12-07,Prague_1
3,6550.0,2020-12-08,Prague_1
4,6910.0,2020-12-09,Prague_1
...,...,...,...
7335,6733.0,2024-03-10,Budapest_1
7336,6492.0,2024-03-11,Budapest_1
7337,6661.0,2024-03-12,Budapest_1
7338,6843.0,2024-03-13,Budapest_1


In [6]:
df_lagged = create_lagged_features(df, "orders", lags=14)
df_lagged

,orders,date,warehouse,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14
14,7868.0,2020-12-19,Prague_1,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0,6584.0,6895.0
15,8128.0,2020-12-20,Prague_1,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0,6584.0
16,8605.0,2020-12-21,Prague_1,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0
17,8875.0,2020-12-22,Prague_1,8605.0,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0
18,8968.0,2020-12-23,Prague_1,8875.0,8605.0,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,6733.0,2024-03-10,Budapest_1,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0,6363.0
7336,6492.0,2024-03-11,Budapest_1,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0
7337,6661.0,2024-03-12,Budapest_1,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0
7338,6843.0,2024-03-13,Budapest_1,6661.0,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0


In [7]:
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', seed=42)
xgb_pipe = make_pipeline(xgb_regressor)

In [23]:
date_encoder = DateToOrdinal()
df_lagged["date"] = date_encoder.fit_transform(df_lagged["date"].copy())

In [21]:
df_lagged

,orders,date,warehouse,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14
14,7868.0,0,Prague_1,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0,6584.0,6895.0
15,8128.0,1,Prague_1,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0,6584.0
16,8605.0,2,Prague_1,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0,7030.0
17,8875.0,3,Prague_1,8605.0,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0,6550.0
18,8968.0,4,Prague_1,8875.0,8605.0,8128.0,7868.0,8072.0,7538.0,7429.0,7243.0,7010.0,6844.0,7165.0,7790.0,7228.0,6910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,6733.0,1173,Budapest_1,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0,6363.0
7336,6492.0,1174,Budapest_1,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0
7337,6661.0,1175,Budapest_1,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0
7338,6843.0,1176,Budapest_1,6661.0,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0


In [11]:
warehouses = df_lagged["warehouse"].unique().tolist()

for warehouse in warehouses:
    warehouse_data = df_lagged[df_lagged["warehouse"] == warehouse]

    warehouse_data_X = warehouse_data.drop(columns=["orders", "warehouse"])
    warehouse_data_y = warehouse_data["orders"]

    final_score = abs(cross_val_score(xgb_pipe, warehouse_data_X, warehouse_data_y, cv=5, scoring="neg_mean_absolute_percentage_error").mean()) * 100

    print(warehouse, final_score)
    print()

Prague_1 6.240961325474305

Brno_1 7.303592055891049

Prague_2 7.092069255764989

Prague_3 7.819161557821501

Munich_1 29.296961560827672

Frankfurt_1 7.213988773178319

Budapest_1 5.5839188282123935



### We see that we have much better values with lagged features, expecially with 14 days ! :)

### TODO: 
* Find best lag-size
* Gridsearch best model parameters

In [13]:
warehouse_Prague_1 = df_lagged[df_lagged["warehouse"] == "Prague_1"]

Prague_1_data_X = warehouse_data.drop(columns=["orders", "warehouse"])
Prague_1_data_y = warehouse_data["orders"]

In [15]:
# Define the parameter grid
param_grid = {
    'xgbregressor__learning_rate': [0.01, 0.05, 0.1],
    'xgbregressor__n_estimators': [100, 200, 500],
    'xgbregressor__max_depth': [3, 5, 7],
    'xgbregressor__min_child_weight': [1, 3, 5],
    #'xgbregressor__subsample': [0.6, 0.8, 1.0],
    #'xgbregressor__colsample_bytree': [0.6, 0.8, 1.0],
    'xgbregressor__gamma': [0, 0.1, 0.2],
    #'xgbregressor__reg_alpha': [0, 0.01, 0.1],
    #'xgbregressor__reg_lambda': [1, 1.5, 2]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(xgb_pipe, param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', verbose=1, n_jobs=-1)

# Assuming X_train and y_train are your training data
grid_search.fit(Prague_1_data_X, Prague_1_data_y)

# Output the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


Best parameters found:  {'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.01, 'xgbregressor__max_depth': 5, 'xgbregressor__min_child_weight': 5, 'xgbregressor__n_estimators': 500}
Best score:  -0.044825297384936826


In [16]:
best_model = grid_search.best_estimator_

In [17]:
for warehouse in warehouses:
    warehouse_data = df_lagged[df_lagged["warehouse"] == warehouse]

    warehouse_data_X = warehouse_data.drop(columns=["orders", "warehouse"])
    warehouse_data_y = warehouse_data["orders"]

    final_score = abs(cross_val_score(best_model, warehouse_data_X, warehouse_data_y, cv=5, scoring="neg_mean_absolute_percentage_error").mean()) * 100

    print(warehouse, final_score)
    print()

Prague_1 5.807119477252454

Brno_1 6.53495756473059

Prague_2 6.558041539649724

Prague_3 6.4023933968812035

Munich_1 26.131981615514167

Frankfurt_1 6.786399010126503

Budapest_1 4.482529738493683



In [24]:
filename1 = "test.csv"
df_test = pd.read_csv(os.path.join("..", data_folder, filename1))#, parse_dates=True, index_col="date")

df_test = df_test[["date", "warehouse"]]

df_test["date"] = date_encoder.transform(df_test["date"].copy())

In [25]:
df_test

,date,warehouse
0,1193,Prague_1
1,1194,Prague_1
2,1195,Prague_1
3,1196,Prague_1
4,1197,Prague_1
...,...,...
392,1249,Budapest_1
393,1250,Budapest_1
394,1251,Budapest_1
395,1252,Budapest_1


In [ ]:
# def create_lagged_features_1(df, value, lags):
#     for lag in range(1, lags + 1):
#         df[f't-{lag}'] = df[value].shift(lag)
#     df.dropna(inplace=True)
#     return df


# def create_lagged_features_for_prediction_iterativly(model, df_train_with_ordinal_time, df_test_with_ordinal_time):
#     create_lagged_features_1(df, "orders", lags=14)


In [29]:
for warehouse in warehouses:
    warehouse_data_train = df_lagged[df_lagged["warehouse"] == warehouse]

    warehouse_data_test = df_test[df_test["warehouse"] == warehouse]

    warehouse_data_X = warehouse_data_train.drop(columns=["orders", "warehouse"])
    warehouse_data_y = warehouse_data_train["orders"]

    print(warehouse_data_X.tail())

    #best_model.fit(warehouse_data_X, warehouse_data_y)

    #best_model.predict(warehouse_data_test)
    break

      date      t-1      t-2      t-3      t-4      t-5      t-6      t-7  \
1188  1174  10250.0  10344.0  11917.0   9754.0   9283.0   9316.0   9520.0   
1189  1175   9866.0  10250.0  10344.0  11917.0   9754.0   9283.0   9316.0   
1190  1176   9710.0   9866.0  10250.0  10344.0  11917.0   9754.0   9283.0   
1191  1177   9628.0   9710.0   9866.0  10250.0  10344.0  11917.0   9754.0   
1192  1178  10056.0   9628.0   9710.0   9866.0  10250.0  10344.0  11917.0   

         t-8      t-9     t-10     t-11     t-12     t-13     t-14  
1188  9900.0  10432.0  11683.0   9628.0   9078.0   9074.0   9776.0  
1189  9520.0   9900.0  10432.0  11683.0   9628.0   9078.0   9074.0  
1190  9316.0   9520.0   9900.0  10432.0  11683.0   9628.0   9078.0  
1191  9283.0   9316.0   9520.0   9900.0  10432.0  11683.0   9628.0  
1192  9754.0   9283.0   9316.0   9520.0   9900.0  10432.0  11683.0  
